Find nearby Scheduled Monuments in England by postcode.

This code will find the nearest 5 scheduled monuments to the centre point of any given English postcode (just the first half!).


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
pd.set_option('display.max_colwidth', 175)
import folium
import BackgroundFunctions as bf

In [ ]:
#help(monument_finder)

In [2]:
pcode= 'NE1'
bf.monument_finder_map(pcode)

C:\Users\inkyt\anaconda3\envs\egm722\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
